# Création du dataset

## Importation des packages

In [4]:
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile
import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml
pd.set_option('display.expand_frame_repr', False)

## Vérification des versions

In [8]:
print('Python  version {}'.format(sys.version))
print('Numpy   version {}'.format(np.__version__))
print('Pandas  version {}'.format(pd.__version__))

Python  version 3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]
Numpy   version 1.20.3
Pandas  version 1.3.2
